# importing the required libraries 

In [ ]:
import re
import pandas as pd
import numpy as np
import emoji
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
nltk.download('vader_lexicon')

# definition of functions

In [ ]:
# Function to check if a line starts with date and time
def date_time(s):
    pattern = r'^([0-9]{1,2}/[0-9]{1,2}/[0-9]{2,4}), ([0-9]{1,2}:[0-9]{2})\u202F?(AM|PM|am|pm)? - '
    result = re.match(pattern, s)
    return bool(result)
    
# Function to find if the line contains a contact name and message
def find_contact(s):
    s=s.split(":")
    if len(s)==2:
        return True
    else:
        return False
        
def get_message(line):
    split_line = line.split(' - ')
    datetime_part = split_line[0]
    date, time = datetime_part.split(', ')
    
    author = None 
    message = ""  

    if len(split_line) > 1:
        message_part = " ".join(split_line[1:])
        if find_contact(message_part):
            split_message = message_part.split(": ")
            author = split_message[0]
            message = " ".join(split_message[1:])
        else:
            message = message_part  

    return date, time, author, message

In [ ]:
# Processing the chat file
data = []
conversation ='your whatsaap chat file.txt'  #droup your file.txt
with open(conversation, encoding="utf-8") as fp:
    fp.readline()  # Skip the first line if it is a header

    message_buffer = []
    date, time, author = None, None, None
    
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        
        if date_time(line):
            if len(message_buffer)>0:            
                data.append([date, time, author, ' '.join(message_buffer)])
            message_buffer.clear()
            
            date, time, author, message = get_message(line)
            message_buffer.append(message)
        else:
            message_buffer.append(line)

# analyses the sentiments

In [ ]:
# Create DataFrame
df = pd.DataFrame(data, columns=["Date", "Time", "author", "Message"])
df['Date'] = pd.to_datetime(df['Date'])
df = df.dropna()

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()
df["positive"] = [sentiments.polarity_scores(i)["pos"] for i in df["Message"]]
df["negative"] = [sentiments.polarity_scores(i)["neg"] for i in df["Message"]]
df["neutral"] = [sentiments.polarity_scores(i)["neu"] for i in df["Message"]]

df.head(50) 

In [ ]:
x=df["positive"].sum()
y=df["negative"].sum()
z=df["neutral"].sum()

def score(a,b,c):
    if (a>b) and (a>c):
        print ("Positive")
    if (b>a) and(b>c):
        print("Negative")
    if (c>a) and (c>b):
        print("Neutral")
        
score(x,y,z)
        